# Limpeza dos dados no dataset Titanic

## Importando as bibliotecas necessárias

In [ ]:
from matplotlib import pyplot as plt
from sklearn.utils import resample
import seaborn as sns
import pandas as pd
import numpy as np

## Link para o dataset principal

[Dataset principal - main.ipynb](./main.ipynb)

## Importando o Dataset

In [ ]:
df_titanic_raw = pd.read_csv("./databases/raw/titanic.csv", sep = ",", index_col = "PassengerId")

## Breve apresentacao do dataset

In [ ]:
df_titanic_raw.sample(10)

In [ ]:
df_titanic_raw.info()

No dataset Titanic, podemos ver que há dados faltantes em diversas colunas.

Será realizado um análise e um possível limpeza nesses dados.

## Limpeza dos dados no dataset Titanic

In [ ]:
df_titanic_raw.isna().sum()

### Verificando colunas problemáticas

In [ ]:
print(df_titanic_raw["Age"].head(10))
print(df_titanic_raw["Cabin"].head(10))
print(df_titanic_raw["Embarked"].head(10))

Foi após análise visual, foi decidido:
- deletar a coluna "Cabin";
- preencher a coluna "Embarked";
    - Utilizando a moda.
- preenchar a coluna "Age".
    - Utilizando a mediana.

### Realizando mudanças no dataset

In [ ]:
df_titanic_raw["Embarked"].mode()

In [ ]:
df_titanic_processed = df_titanic_raw.drop(columns = ["Cabin"], axis = "columns")
df_titanic_processed["Age"] = df_titanic_processed["Age"].fillna(df_titanic_processed["Age"].median())
df_titanic_processed["Embarked"] = df_titanic_processed["Embarked"].fillna("S")
df_titanic_processed.isna().sum()

### Descartando colunas vistas como inúteis para a análise dos dados

In [ ]:
colunas_para_descartar = [
    "Parch",
    "Ticket"
]

In [ ]:
df_titanic_processed.drop(columns = colunas_para_descartar, axis = "column", inplace = True)

### Mudanças nos tipos do dado do dataset Titanic

In [ ]:
df_titanic_processed.info()

As colunas:
- Survived;
- Pclass;
- Sex;
- Embarked.

Serão classificadas como tipo "categórico"(category).

As colunas:
- SibSp;
- Age.

Serão classificadas como tipo "inteiro"(uint8).

A coluna "Fare" será classificada como tipo ponto flutuante (float32).

A coluna "Name" será classificada como tipo alfanumérico (str).

In [ ]:
df_titanic_processed["Survived"]    = df_titanic_processed["Survived"].astype("category")
df_titanic_processed["Pclass"]      = df_titanic_processed["Pclass"].astype("category")
df_titanic_processed["Name"]        = df_titanic_processed["Name"].astype("string")
df_titanic_processed["Sex"]         = df_titanic_processed["Sex"].astype("category")
df_titanic_processed["Age"]         = df_titanic_processed["Age"].astype("uint8")
df_titanic_processed["SibSp"]       = df_titanic_processed["SibSp"].astype("uint8")
df_titanic_processed["Fare"]        = df_titanic_processed["Fare"].astype("float32")
df_titanic_processed["Embarked"]    = df_titanic_processed["Embarked"].astype("category")

#### Verificando mudança

In [ ]:
df_titanic_processed.info()

### Mudanças no valores da coluna "Sex"

In [ ]:
df_titanic_processed["Sex"].unique()

Para melhor manipulação do dataset, foi escolhido manter os dados categóricos seguindo a
convenção de números inteiros, pequenos e positivos, como é possível notar nas colunas
"Survived" e "Pclass".

Portanto será realizado uma manipulação dos dados, de forma que:
- ` female ` será substituido pelo número ` 0 `;
- ` male ` será substituido pelo número ` 1 `.

In [ ]:
novos_valores_Sex = {
    "female": 0,
    "male": 1
}

In [ ]:
df_titanic_processed["Sex"] = df_titanic_processed["Sex"].map(novos_valores_Sex)
df_titanic_processed["Sex"].unique()

### Mudanças no valores da coluna "Embarked"

In [ ]:
df_titanic_processed["Embarked"].unique()

Para melhor manipulação do dataset, foi escolhido manter os dados categóricos seguindo a
convenção de números inteiros, pequenos e positivos, como é possível notar nas colunas
"Sex", "Survived" e "Pclass".

Portanto será realizado uma manipulação dos dados, de forma que:
- ` C ` será substituido pelo número ` 0 `;
- ` Q ` será substituido pelo número ` 1 `;
- ` S ` será substituido pelo número ` 2 `.

In [ ]:
novos_valores_Embarked = {
    "C": 0,
    "Q": 1,
    "S": 2
}

In [ ]:
df_titanic_processed["Embarked"] = df_titanic_processed["Embarked"].map(novos_valores_Embarked)
df_titanic_processed["Embarked"].unique()

### Verificando se os dados da coluna "Pclass" estão balanceados

In [ ]:
df_titanic_processed["Pclass"].value_counts()

Como podemos ver acima, os dados estão muito desbalanceados.

Para uma melhor análise, será preciso reabalancear os dados, ou para mais ou para menos.
Se balancearmos para menos, perderemos uma quantidade enorme de dados, então foi escolhido balancear para mais.

#### Gerando vários datasets para balancear os dados

##### Hiper parametros

In [ ]:
tamanho_das_classes = df_titanic_processed[df_titanic_processed["Pclass"] == 3].shape[0]
semente_randomica = 42

##### Realizando a criação

In [ ]:
df_Pclass_1 = df_titanic_processed[df_titanic_processed["Pclass"] == 1]
df_Pclass_2 = df_titanic_processed[df_titanic_processed["Pclass"] == 2]
df_Pclass_3 = df_titanic_processed[df_titanic_processed["Pclass"] == 3]

##### Realizando a balanceamento

In [ ]:
df_Pclass_1 = resample(df_Pclass_1,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)
df_Pclass_2 = resample(df_Pclass_2,
                       replace = True,
                       n_samples = tamanho_das_classes,
                       random_state = semente_randomica)

#### Unindo os datasets em um só

In [ ]:
df_titanic_processed_balanceado = pd.concat([df_Pclass_3, df_Pclass_1, df_Pclass_2])
df_titanic_processed_balanceado["Pclass"].value_counts()

## Salvando o dataset tratado em um pickle (.pkl)

In [ ]:
df_titanic_processed_balanceado.to_pickle("./databases/processed/titatic_processado.pkl")